In [124]:
from smartcard import ATR
from smartcard.System import readers
from smartcard.CardConnection import CardConnection

In [125]:
rarr=readers()
if len(rarr) == 0:
    print("No readers detected")
else:
    reader = rarr[0]
    print(reader)

Generic Smart Card Reader Interface


In [126]:
protocol = CardConnection.T1_protocol
connection = reader.createConnection()
connection.connect(protocol)

In [127]:
atr=bytes(connection.getATR())
atr.hex()

'3bdc18ff8191fe1fc38073c821136605036351000250'

In [128]:
def select(appletID):
    data = list(bytes.fromhex(appletID))
    # Select:
    # CLA = 0x00
    # INS = 0xA4
    # P1 = 0x04
    # P2 = 0x00
    # Data = the instance AID
    cmd = [0x00, # CLA
              0xA4, # INS
              0x04, # P1
              0x00, # P2
              len(data), # Lc (content length)
          ] + data + [0x00]
    print(bytes(cmd).hex().upper())
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [129]:
select("B00B5111CB01")

00A4040006B00B5111CB0100


b''

In [130]:
def request(APDU):
    cmd = list(bytes.fromhex(APDU))
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [138]:
# get pubkey - private key is generated when applet is installed
import time
t0 = time.time()
print(request("B0A1000000").hex())
t1 = time.time()
print(t1-t0)

04ccabd447644da561ff4a70edb188f5acf123da7373d29f26bd43e4736319c30b4ef3ddd2fa709587966fa7a7bee24981c5cc5a1ea104d9059d6e2f16f029f827
0.32341670989990234


In [139]:
# get random data
rand = request("B0A2000000")
rand.hex()

'c3c67352c67a4877fa53faba9bf719720cc8f9cf43c9fea7b01bbd8659f88431'

In [140]:
# static data for any other APDU
data = request("B000000000")
data

b'Memory cards are not safu so what?'

In [123]:
connection.disconnect()